# MIHCSME OMERO Package Demo

## 1. Setup and Imports

In [ ]:
from pathlib import Path
import json
from pprint import pprint

from mihcsme_py import parse_excel_to_model, upload_metadata_to_omero
from mihcsme_py.models import (
    MIHCSMEMetadata,
    AssayCondition,
    InvestigationInformation,
    StudyInformation,
    AssayInformation,
)

# Optional: For nice table display
import pandas as pd

print("✅ Imports successful!")

## 2. Parse MIHCSME Excel File

Let's parse an Excel file containing MIHCSME metadata.

In [ ]:
# Path to your MIHCSME Excel file
excel_path = Path("../MIHCSME Template_example.xlsx")


In [ ]:
# Parse the Excel file into a Pydantic model
metadata = parse_excel_to_model(excel_path)

print(f"✅ Successfully parsed metadata!")
print(f"   Number of wells: {len(metadata.assay_conditions)}")
print(f"   Number of reference sheets: {len(metadata.reference_sheets)}")

## 3. Inspect Metadata Structure

The parsed metadata is a fully typed Pydantic model with four main sections:
- Investigation Information
- Study Information
- Assay Information
- Assay Conditions (per-well data)

### 3.1 Investigation Information

In [ ]:
print("📋 Investigation Information Groups:")
for group_name, fields in metadata.investigation_information.groups.items():
    print(f"\n{group_name}:")
    for key, value in fields.items():
        print(f"  {key}: {value}")

### 3.2 Study Information

In [ ]:
print("🔬 Study Information:")
for key, value in metadata.study_information.groups.items():
    print(f"  {key}: {value}")

### 3.3 Assay Information

In [ ]:
print("🧪 Assay Information:")
for key, value in metadata.assay_information.groups.items():
    print(f"  {key}: {value}")

### 3.4 Assay Conditions (Well-level Data)

Let's look at the first few wells and their conditions.

In [ ]:
print(f"🔬 Total wells with conditions: {len(metadata.assay_conditions)}\n")

# Show first 5 wells
print("First 5 wells:")
for condition in metadata.assay_conditions[:5]:
    print(f"\nPlate: {condition.plate}, Well: {condition.well}")
    print(f"Conditions: {condition.conditions}")

### 3.5 Display as DataFrame

Convert well conditions to a pandas DataFrame for easy viewing.

In [ ]:
# Convert assay conditions to a list of dicts
df = metadata.to_dataframe()

print(f"📊 Assay Conditions DataFrame ({len(df)} rows):")
df.head(10)

## 4. Filter and Query Metadata

Programmatically access and filter the metadata.

In [ ]:
# Get all unique plates
plates = {condition["Plate"] for condition in df.to_dict(orient="records")}
print(f"📋 Unique plates: {plates}")

# Filter by plate
if plates:
    first_plate = list(plates)[0]
    plate_conditions = [
        c for c in metadata.assay_conditions if c.plate == first_plate
    ]
    print(f"\n🔍 Wells in {first_plate}: {len(plate_conditions)}")

In [ ]:
# Filter wells by specific condition
# Example: Find all wells with a specific compound (adjust key name as needed)
condition_key = "Treatment"  # Change this to match your data
condition = {condition[condition_key] for condition in df.to_dict(orient="records")}

pprint(condition)

## 5. Modify Assay Conditions as dataframe and add back to metadata

In [ ]:
df = metadata.to_dataframe()
# Make all treatment names lowercase    
df["treatment"] = df["Treatment"].str.lower()

metadata = metadata.update_conditions_from_dataframe(df)


## 6. Export to JSON

Export the metadata to JSON format for storage or sharing.

In [ ]:
# Export as JSON (Pydantic's native format)
output_json = Path("../metadata_export.json")

with open(output_json, "w") as f:
    json.dump(metadata.model_dump(), f, indent=2)

print(f"✅ Exported metadata to: {output_json}")
print(f"   File size: {output_json.stat().st_size / 1024:.1f} KB")

In [ ]:
# Preview the JSON structure (first 30 lines)
if output_json.exists():
    with open(output_json) as f:
        lines = f.readlines()[:30]
        print("📄 JSON Preview (first 30 lines):")
        print("=" * 60)
        print("".join(lines))
        if len(f.readlines()) > 30:
            print("...")
else:
    print(f"⚠️  File not found: {output_json.absolute()}")
    print("   Please run the previous cell first to export the JSON.")

## 7. Convert to OMERO Format

Convert the Pydantic model to the dictionary format used by OMERO.

In [ ]:
# Convert to OMERO dict format
omero_dict = metadata.to_omero_dict()

print("📦 OMERO Dictionary Structure:")
print(f"   Keys: {list(omero_dict.keys())}")
print(f"\n   Investigation keys: {list(omero_dict.get('InvestigationInformation', {}).keys())[:5]}")
print(f"   Study keys: {list(omero_dict.get('StudyInformation', {}).keys())[:5]}")
print(f"   Assay keys: {list(omero_dict.get('AssayInformation', {}).keys())[:5]}")
print(f"   Assay conditions count: {len(omero_dict.get('AssayConditions', []))}")

## 7. Create Metadata Programmatically

You can also create metadata objects from scratch in Python.

In [ ]:
# Create a simple metadata object
custom_metadata = MIHCSMEMetadata(
    investigation_information=InvestigationInformation(
        groups={
            "Project": {
                "Investigation Title": "Demo Investigation",
                "Investigation Description": "Created programmatically",
            }
        }
    ),
    study_information=StudyInformation(
        fields={
            "Study Title": "Demo Study",
            "Study Description": "Example study",
        }
    ),
    assay_information=AssayInformation(
        fields={
            "Assay Title": "Demo Assay",
            "Assay Type": "High Content Screening",
        }
    ),
    assay_conditions=[
        AssayCondition(
            plate="DemoPlate",
            well="A1",  # Automatically normalized to "A01"
            conditions={
                "Compound": "DMSO",
                "Concentration": "0.1%",
                "Treatment Time": "24h",
            },
        ),
        AssayCondition(
            plate="DemoPlate",
            well="A2",
            conditions={
                "Compound": "Drug X",
                "Concentration": "10 μM",
                "Treatment Time": "24h",
            },
        ),
        AssayCondition(
            plate="DemoPlate",
            well="B1",
            conditions={
                "Compound": "Drug X",
                "Concentration": "1 μM",
                "Treatment Time": "24h",
            },
        ),
    ],
)

print("✅ Created custom metadata object")
print(f"   Wells: {len(custom_metadata.assay_conditions)}")
print(f"\n   Well names (auto-normalized): {[c.well for c in custom_metadata.assay_conditions]}")

## 8. Validate Well Format

Pydantic automatically validates well names and normalizes them.

In [ ]:
# Valid well formats (will be normalized)
valid_wells = ["A1", "A01", "B12", "P48"]

for well in valid_wells:
    condition = AssayCondition(
        plate="Test",
        well=well,
        conditions={},
    )
    print(f"Input: '{well}' → Normalized: '{condition.well}'")

In [ ]:
# Invalid well formats (will raise ValidationError)
invalid_wells = ["Q1", "A49", "AA1", "1A", "A0"]

print("\n❌ Testing invalid well formats:")
for well in invalid_wells:
    try:
        condition = AssayCondition(
            plate="Test",
            well=well,
            conditions={},
        )
        print(f"  {well}: ✅ Valid (unexpected!)")
    except ValueError as e:
        print(f"  {well}: ❌ Invalid - {str(e)[:80]}...")

## 9. Upload to OMERO

⚠️ **This section requires a live OMERO connection.** Skip if you don't have access to an OMERO server.

To upload metadata to OMERO, you need:
- OMERO server URL
- Username and password
- Target Screen ID or Plate ID

In [ ]:
# OMERO connection parameters (update these!)
OMERO_HOST = "localhost"  # Change this
OMERO_USER = "test"  # Change this
OMERO_PASSWORD = ""  # Change this (or use getpass)

# Target for upload
TARGET_TYPE = "Screen"  # or "Plate"
TARGET_ID = 3015  # Change this to your Screen/Plate ID

print("⚠️  OMERO Upload Configuration:")
print(f"   Host: {OMERO_HOST}")
print(f"   User: {OMERO_USER}")
print(f"   Target: {TARGET_TYPE} ID {TARGET_ID}")
print("\n   ⚠️  Update these values before running!")

In [ ]:
# Uncomment to run the upload
import ezomero
# Connect to OMERO
print("🔌 Connecting to OMERO...")
conn = ezomero.connect(
    host=OMERO_HOST,
    user=OMERO_USER,
    password=OMERO_PASSWORD,
    secure=True,
)
print("✅ Connected!")

# Upload metadata
print(f"\n📤 Uploading metadata to {TARGET_TYPE} {TARGET_ID}...")
result = upload_metadata_to_omero(
    conn=conn,
    metadata=metadata,
    target_type=TARGET_TYPE,
    target_id=TARGET_ID,
    namespace="MIHCSME",
    replace=False,  # Set to True to replace existing annotations
)

# Display results
print("\n📊 Upload Results:")
print(f"   Status: {result['status']}")
print(f"   Wells processed: {result['wells_processed']}")
print(f"   Wells succeeded: {result['wells_succeeded']}")
print(f"   Wells failed: {result['wells_failed']}")

if result['errors']:
    print(f"\n Errors encountered:")
    for error in result['errors'][:5]:  # Show first 5 errors
        print(f"   - {error}")

# Close connection
conn.close()
print("\n✅ Upload complete!")


print("ℹ️  Upload code commented out. Uncomment to run.")